<center><h1><font face="vollkorn" color=navy> Price Errors Between Invoices and POs</font></h1></center>
<center><h2> Are we systematically overpaying? </h2></center><br>
<p>
<img src="http://www.ncseprojects.org/files/images/berkeley_logo_300.gif" style="float: left; width: 15%; margin-left: 30%;">

<img src="http://www.ucsf.edu/sites/all/themes/ucsf/logo.png" style="float: right; width: 20%; margin-right: 30%; margin-bottom: 10%;">
<p style="clear: both;">

<center><h1><font face="vollkorn" color=navy>Contributors:</font></h1></center>
<p>
<center><h2>Andrew G. Clark</h2></center>

<font face="vollkorn"><p><b>Purpose:</b><br>
This book explores the accuracy of pricing invoiced to UCSF and UC Berkeley versus the Purchase Order. 

<p><b>Background:</b> <br>
UCSF issues Purchase Orders (binding legal contracts) to suppliers as a "promise" to pay for goods and services. The PO serves as official notice to the supplier of the goods/services we wish to buy, along with the price we are authorizing for payment, and any other terms and conditions we require.  It is generally expected that the supplier will either fill the purchase order and send UCSF the goods/services ordered or will call UCSF to identify a problem with the order.  
<p>
In Febuary 2015, UCSF BearBuy Operations was notified by a key stakeholder that they belived we were being systematically over-invoiced.  That is to say, suppliers were issuing invoices to UCSF for dollar amounts greater than the PO authorized.  As a result, this was causing a great deal of work for this stakeholder who was carefully reconciling her budgets.

<p><b>Assumptions:</b><br>
+ PO Line Extended Price vs. Invoice Line Extended Price is correct identifier of price error.
+ The invoice full extract from BearBuy(s) is the right dataset to use.
+ BearBuy PO number is a reasonable approximation for time.

<p><b>To Do:</b><br>
+ I should identify the blankets and then mark them in the initial data set in a new column.

<font face="vollkorn"><h2>Read in data and pre-process it</h2>
<p>The BearBuy Invoice extract includes a number of columns unnecessary for this analysis.  This section of code reads in the data, creates a list of important columns, then removes all columns not on the list.

In [1]:
%pylab inline
import pandas as pd
import numpy as np

#Read in the data
RawData = pd.read_csv('/home/analyst/Desktop/invoices/all.csv', low_memory=False, thousands=',')


#The UCSF file has different headers to describe the same data.  Rename UCSF Headers to match UCB Headers.
try:
    RawData.rename(columns={'PO #':'PO No', 
                            'Extended Price1':'Invoice Line Extended Price',
                            'Extended Price2':'PO Line Extended Price',
                            'PO Line #':'PO Line No',
                            'Voucher #':'Voucher No',
                            'Invoice System Creation Date':'Invoice System Created Date',
                            'Home Department Name':'Department Name',
                            'Invoice Type':'Voucher Type'
                           }, inplace=True)
except:
    pass

#Eliminate columns not used in the analysis [uses the Berkeley file name convention]
keep_columns = ['PO No',
                'PO Line No',
                'PO Line Unit Price',
                'PO Line Extended Price',
                'Supplier Name',
                'Manufacturer',
                'Department Name',
                'Invoice Owner',
                'Item Type',
                'Form Type',
                'Voucher No',
                'Voucher Source',
                'Voucher Type',
                'Invoice Line Unit Price',
                'Invoice Line Extended Price',
                'Voucher Status',
                'Workflow Status',
                'Line Status'
               ]

invoices = RawData[keep_columns];

Populating the interactive namespace from numpy and matplotlib


<font face="vollkorn"><h2>Filter the data and add PO features</h2>
<p>The dataset contains far more types of documents than "regular" invoices.  As such some additional filtering is required.  Specifically:
* Filter non-PO data: We eliminate anything not starting with a "B".
* Filter unprocessed invoices: The data includes a host of invoices which have not been approved by campus accounts payable.  These are removed.
* Filter credit memos: We constrain our focus to just invoices and not credit memos or other documents in the data.
* Filter Blanket POs: Blanket POs are PO Lines that are used to pay for multiple invoices.  There is no apriori specified buy when the Blanket is created, rather, these are authorizations for campus officials to spend fixed dollar amounts with a specific supplier.  This typically makes sense when administration wants to frontload all approvals and authorizations to better streamline the buying activity of researchers, faculty, and other staff.  


In [4]:
#filter out all of the non-invoice invoices...
invoices_filtered = invoices[(invoices['PO No'].str.startswith('B')) &
                              (invoices['Voucher Status'] != 'Rejected') &
                              (invoices['Invoice Line Extended Price'] > 0) &
                              (invoices['Voucher Type'] != 'Credit Memo') &
                              (invoices['Workflow Status'] == 'Complete') &
                              (invoices['Line Status'] == 'Approved')]

#Summarize the data by PO Number and calculate some features
summary = invoices_filtered.groupby(['PO No','PO Line No','Supplier Name']).agg({'PO Line Extended Price': lambda x: x.unique(),
                                                                                 'Invoice Line Extended Price': np.sum,
                                                                                 'Voucher No': lambda x: len(x.unique())
                                                                                 })

#Set the datatypes for the fields to float
summary['Invoice Line Extended Price'] = summary['Invoice Line Extended Price'].replace(',','').astype(float)
summary['PO Line Extended Price'] = summary['PO Line Extended Price'].replace(',','').astype(float)

#Calculate the features of interest
summary['PO Amount Used'] = summary['Invoice Line Extended Price'] / summary['PO Line Extended Price']
summary['PriceDiff'] = summary['Invoice Line Extended Price'] - summary['PO Line Extended Price'] 
summary['PercentDiff'] = summary['PriceDiff'] / summary['PO Line Extended Price']


<font face="vollkorn"><h2>What do the non-blanket PO price errors look like?</h2>
<p>The first fact we notice when analyzing the data is that the price errors are centered around zero.  Also, we also notice that we are running a rather low error rate on PO Lines.

In [21]:
#Remove the blanket POs by creating a filter
NonBlanket_filter = summary[(summary['Voucher No'] <= 2) &
                      (summary['PO Amount Used'] >= .70) &
                      (summary['PercentDiff'] >= -.40) &
                      (summary['PercentDiff'] <= .40) 
                     ].reset_index()


#Filter the invoices to just the non-blankets
NonBlanket = invoices_filtered[invoices_filtered['PO No'].isin(NonBlanket_filter['PO No'])]

#Add some features
NonBlanket['PriceDiff'] = NonBlanket['Invoice Line Extended Price'] - NonBlanket['PO Line Extended Price']
NonBlanket['PercentDiff'] = NonBlanket['PriceDiff'] / NonBlanket['PO Line Extended Price'];
NonBlankbet['IsError'] = NonBlanket['PercentDiff'] != 0

#maxbin = .4
#minbin = -.4
#binwidth = .05

#Plot a Histogram of all of the non-Zero, Non-Blanket PO Lines
#NonBlanket[NonBlanket['PercentDiff'] != 0].hist('PercentDiff', bins=np.arange(minbin, maxbin + binwidth, binwidth),
#                                                  alpha=.7, figsize=(10,6),histtype='bar',xlabelsize=14, ylabelsize=14)
#plt.xlabel('Percent Price Error', fontsize=14);
#plt.ylabel('Count of PO Lines', fontsize=14);
#plt.title('Histogram - PO lines vouchered with Price Differences', fontsize=14)
#plt.tight_layout()

#print '%0.2f percent of Non-Blanket POs contain a price error' %round(100*(float(NonBlanket[NonBlanket['PercentDiff'] != 0].count()[1]) / float(NonBlanket.count()[1])),2)


/home/analyst/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/analyst/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/analyst/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas

##HOW DO I CREATE THE COLUMNS ABOVE WITHOUT THROWING THESE ERRORS?

In [6]:
#Which Items Seem to have the most problems
NonBlankets['PriceError'] = NonBlankets['Invoice Line Extended Price'] - NonBlankets['PO Line Extended Price'] 
NonBlankets['IsError'] = NonBlankets['PriceError'] != 0

ItemType = NonBlankets.groupby(['Item Type','IsError']).agg({'PriceError':'count'}).unstack()

ItemType.columns = ItemType.columns.droplevel(0)
ItemType = ItemType.reset_index()
ItemType = ItemType.rename(columns = {ItemType.columns[1]:'NotError',
                                      ItemType.columns[2]:'IsError'
                                      })

ItemType['Percent Error'] = ItemType['IsError'] / (ItemType['IsError'] + ItemType['NotError']) * 100

ItemType

/home/analyst/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/analyst/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


IsError,Item Type,NotError,IsError,Percent Error
0,Contract NonCatalogItem,4550,139,2.964385
1,NonCatalog Product,53718,3201,5.623781
2,Punch-out Product,45250,1012,2.187541
3,SQ Hosted Product,53792,811,1.485266


##Analyze by Supplier

<font face="vollkorn">

In [7]:
#Create a binary indicator of price error
tot['PriceError'] = tot['PercentDiff'] != 0

#Summerize the price error by total lines 
SupplierSummary = tot.groupby(['Supplier Name','PriceError']).agg({'PO Line No': 'count'}).unstack().fillna(0)

#Format the price error table more intuitively and change the name of the columns
SupplierSummary.columns = SupplierSummary.columns.droplevel(0)
SupplierSummary = SupplierSummary.reset_index()
SupplierSummary = SupplierSummary.rename(columns = {SupplierSummary.columns[1]:'NotError',
                                                    SupplierSummary.columns[2]:'IsError'
                                                    })

#Calculate the percent of total lines in Error
SupplierSummary['PercentError'] = np.round(SupplierSummary['IsError'] / (SupplierSummary['NotError'] + SupplierSummary['IsError'])*100,2)

#Print out the table in descending format
SupplierSummary.sort('IsError', ascending=False).head(10)



PriceError,Supplier Name,NotError,IsError,PercentError
1133,CASCADE ORTHOPEDIC SUPPLY INC,1112,351,23.99
4730,SIGMA-ALDRICH INC,6856,88,1.27
4546,SANTA CRUZ BIOTECHNOLOGY INC,467,60,11.39
5534,"VWR INTERNATIONAL, INC.",11697,56,0.48
3069,LIFE TECHNOLOGIES CORPORATION,4985,45,0.89
3947,"PATTERSON DENTAL SUPPLY, INC",1389,41,2.87
586,ASSOCIATED SERVICES CO,246,35,12.46
4333,"REMEL, INC",54,31,36.47
1163,CDW-GOVERNMENT INC.,2668,24,0.89
1690,DIGI-KEY CORPORATION,174,23,11.68
